# Retrieve Stock Prices

## Load Crawled Data and Build Dataframe

### Libraries

In [0]:
!pip install yfinance

In [0]:
import os
from google.colab import drive
import json
import pandas as pd
from datetime import datetime
from collections import Counter, defaultdict
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib
import matplotlib.dates as mdates
from textblob import TextBlob
import yfinance as yf
from pandas_datareader import data as pdr
import pickle
from datetime import timedelta, date, datetime
from tqdm import tqdm

### Mounting (need to copy data to own Google Drive)

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### SIC Alpha Ratings

In [0]:
file_path = "gdrive/My Drive/sic-code-rankings-03-23-2020.csv"

df = pd.read_csv(file_path)
df = df.drop(65)

def split_sic_code(x):
  sic_code = x["Industry Name"].split(" ")[0].split("-")[1]
  return sic_code

df["SIC"] = df.apply(lambda x: split_sic_code(x), axis = 1)
df

,Symbol,Industry Name,Wtd Alpha,Wtd. Alpha Change,# of Stocks,SIC
0,-005E,SIC-54 Food Stores,-2.82,-2.36,9.0,54
1,-001A,SIC-10 Metal Mining,-13.86,-5.69,53.0,10
2,-006E,"SIC-64 Insurance Agents, Brokers, And Service",-18.40,-0.97,14.0,64
3,-008I,SIC-88 Private Households,-21.34,1.02,5.0,88
4,-008C,SIC-82 Educational Services,-21.93,-0.19,34.0,82
...,...,...,...,...,...,...
60,-004G,"SIC-46 Pipelines, Except Natural Gas",-60.52,4.61,13.0,46
61,-003B,SIC-31 Leather And Leather Products,-60.54,-1.03,9.0,31
62,-002J,SIC-29 Petroleum Refining And Related Indust...,-62.48,-0.25,24.0,29
63,-001D,SIC-13 Oil And Gas Extraction,-70.69,-0.56,103.0,13


In [0]:
#df.to_pickle("gdrive/My Drive/k_10_corona.pkl")
#df_no_text = df.drop(columns=["raw_text"])
df.to_csv("gdrive/My Drive/sic_stocks.csv", encoding = 'utf-8', index=False)

## Stock Prices

In [0]:
df_base = pickle.load(open("gdrive/My Drive/k_10_corona.pkl",mode="rb"))
cik = set(df_base.FirmID)
print(len(cik))
cik = [x.split(" ")[0] for x in cik]
cik = list(map(int, cik))

1379


In [0]:
file_path = "gdrive/My Drive/cik_ticker.csv"

df = pd.read_csv(file_path, delimiter= "|").reset_index()
df = df.drop(columns = ["Incorporated", "IRS","Business","index"])
df

,CIK,Ticker,Name,Exchange,SIC
0,1090872,A,Agilent Technologies Inc,NYSE,3825.0
1,4281,AA,Alcoa Inc,NYSE,3350.0
2,1332552,AAACU,Asia Automotive Acquisition Corp,NaN,6770.0
3,1287145,AABB,Asia Broadband Inc,OTC,8200.0
4,1024015,AABC,Access Anytime Bancorp Inc,NaN,6035.0
...,...,...,...,...,...
13732,827056,ZVXI,Zevex International Inc,NaN,3845.0
13733,846475,ZYNX,Zynex Inc,NaN,3845.0
13734,1406796,ZYTC,Zyto Corp,NaN,3841.0
13735,748015,ZZ,Sealy Corp,NYSE,2510.0


In [0]:
df = df[df['CIK'].isin(cik)]
df

,CIK,Ticker,Name,Exchange,SIC
18,706688,AAN,Aarons Inc,NYSE,7359.0
24,1158449,AAP,Advance Auto Parts Inc,NYSE,5531.0
38,825313,AB,Alliancebernstein Holding LP,NYSE,6282.0
47,351569,ABCB,Ameris Bancorp,NASDAQ,6022.0
136,1037676,ACI,Arch Coal Inc,NYSE,1221.0
...,...,...,...,...,...
13618,1347858,XXII,22ND Century Group Inc,OTCBB,2111.0
13647,108985,YORW,York Water Co,NASDAQ,4941.0
13673,877212,ZBRA,Zebra Technologies Corp,NASDAQ,3560.0
13694,109380,ZION,Zions Bancorporation,NASDAQ,6021.0


In [0]:
def get_stock_data(x):

  try:
    company_ticker = yf.Ticker(str(x["Ticker"]))

    ## company information
    company_info = company_ticker.info
    x["fullTimeEmployees"] = int(company_info["fullTimeEmployees"])
    x["sector"] = company_info["sector"]
    x["marketCap"] = company_info["marketCap"]

    ## stock prices
    company_stocks = company_ticker.history(start= "2020-01-30", end= date.today())
    x["stock 2020-01-30"] = company_stocks.iloc[0,3]
    x["volume 2020-01-30"] = company_stocks.iloc[0,4]
    x["stock " + date.today().strftime("%Y-%m-%d")] = company_stocks.iloc[-1,3] 
    x["volume " + date.today().strftime("%Y-%m-%d")] = company_stocks.iloc[-1,4]
    return x

  except: 
    x["fullTimeEmployees"] = np.nan
    x["sector"] = np.nan
    return x

row_list = []

for index, row in tqdm(df.iterrows()):
  row_new = get_stock_data(row)
  row_list.append(row_new)

df_new = pd.DataFrame(row_list)   
#df = df.apply(lambda x: get_stock_data(x), axis = 1)

141it [02:53,  1.67s/it]

- CHTR: No data found for this date range, symbol may be delisted


151it [03:11,  1.74s/it]

- CLH: No data found for this date range, symbol may be delisted


152it [03:12,  1.69s/it]

...

763it [20:32,  1.94s/it]

- TNC: No data found for this date range, symbol may be delisted


808it [21:49,  1.87s/it]

- UTX: No data found for this date range, symbol may be delisted


864it [23:19,  1.62s/it]


In [0]:
df_new.to_csv("gdrive/My Drive/sic_stocks.csv", encoding = 'utf-8', index=False)